In [23]:
import numpy as np
from susy_qm import create_matrix
from functools import reduce

In [122]:
# Parameters
N = 2     # Number of sites
cutoff = 8    # Basis cutoff
a = 1.0       # Lattice spacing

In [123]:
def create_shift_matrix(num_sites, direction):
 
    S = np.zeros((num_sites, num_sites), dtype=np.complex128)
    for i in range(num_sites - 1):
        if direction == 'forward':
            S[i, i + 1] = 1
        elif direction == 'backward':
            S[i + 1, i] = 1
    return S

In [124]:
q = create_matrix(cutoff, 'q')
p = create_matrix(cutoff, 'p')
S = create_shift_matrix(cutoff, direction='forward')
S_dagger = create_shift_matrix(cutoff, direction='backward')

In [125]:
def tensor_identity(cut_off, site, total_sites, operator):
    """Constructs a tensor product operator for a specific site."""
    I = np.eye(cut_off, dtype=np.complex128)
    operators = [I] * total_sites
    operators[site] = operator
    return reduce(np.kron, operators) 

In [141]:
##def construct_hamiltonian(N, cutoff, a, V, dV):
       
# Initialize the Hamiltonian
dim = cutoff ** N  # Total Hilbert space dimension
H = np.zeros((dim, dim), dtype=np.complex128)

for n in range(N):

    # Bosonic terms
    I_b = np.eye(dim)

    q_n = tensor_identity(cutoff, n, N, q)
    p_n = tensor_identity(cutoff, n, N, p)
    
    # Kinetic term
    H += np.matmul(p_n,p_n) / (2 * a)
    
    # Potential term
    V = q_n
    VP = I_b
    H += (a / 2) * np.matmul(V,V)

    # Subtract the constant offset
    H -= 0.5 * I_b

    # Gradient and interaction terms
    if 0 < n < N - 1:
        print('a')
        q_np1 = tensor_identity(cutoff, (n + 1) % N, N, q)  # (n+1)-th site (periodic boundary)
        q_nm1 = tensor_identity(cutoff, (n - 1) % N, N, q)  # (n-1)-th site (periodic boundary)
    
        gradient = (q_np1 - q_nm1) / (2 * a)
        H += (a / 2) * np.matmul(gradient,gradient)
        H += a * np.matmul(V,gradient)
    
    # Fermionic terms
    if 0 < n < N - 1:
        print('b')
        fermion_term = (-1) ** n * np.matmul(VP, (tensor_identity(cutoff, n, N, S), tensor_identity(cutoff, n, N, S_dagger))) - 0.5 * I_b
        H += fermion_term
    
        # Fermionic hopping
        fermion_hopping = np.matmul(tensor_identity(cutoff, n, N, np.eye(cutoff)), tensor_identity(cutoff, n + 1, N, np.eye(cutoff)))
        H += fermion_hopping / (2 * a)
    
#return H


In [142]:
# Display the Hamiltonian
print("Hamiltonian shape:", H.shape)
print(H.real)


Hamiltonian shape: (64, 64)
[[-2.22044605e-16  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  2.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  8.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   9.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  6.00000000e+00]]


In [143]:
np.sort(np.linalg.eig(H)[0].real)[:6]

array([-2.22044605e-16,  1.00000000e+00,  1.00000000e+00,  2.00000000e+00,
        2.00000000e+00,  2.00000000e+00])